Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give read access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


# Downloads

In [1]:
## DONT CHANGE THIS CELL
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip

--2021-10-11 18:00:58--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip [following]
--2021-10-11 18:00:59--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217313 (212K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 212.22K  95.4KB/s    in 2.2s    

2021-10-11 18:01:02 (95.4 KB/s) - ‘data.zip’ saved [217313/217313]



In [2]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/train/
  inflating: data/train/train.gold.txt  
  inflating: data/train/train.data.txt  
   creating: data/validation/
  inflating: data/validation/validation.data.txt  
  inflating: data/validation/validation.gold.txt  


# Imports/Helpers

In [3]:
## Import relevant packages

import os
import logging
import pandas as pd
import numpy as np

import torch
#from torch.utils.data import Dataset, DataLoader
import torch.optim as O
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

import torchtext
from torchtext import datasets, vocab
from torchtext.legacy import data
from torchtext.vocab import GloVe

from argparse import ArgumentParser
from pdb import set_trace
from typing import Dict, Tuple, List, Union, Optional
from tqdm import tqdm
import time, datetime

import random


In [4]:
# set seeds
random.seed(772)
torch.manual_seed(772)
torch.cuda.manual_seed(772)

In [5]:
## Various utility functions

"""checking arguments"""
def check_args(args):
	# --result_dir
	check_folder(os.path.join(args.out_dir, args.models_dir))
	check_folder(args.tmp_dir)

	# --epoch
	try:
			assert args.epochs >= 1
	except:
			print('number of epochs must be larger than or equal to one')

	# --batch_size
	try:
			assert args.batch_size >= 1
	except:
			print('batch size must be larger than or equal to one')
	 
	# --data_path
	assert os.path.isfile(args.data_path), "data_path not found"
	assert os.path.isfile(args.gold_path), "gold_path not found"

	# --data_split
	args.data_split = list(map(float, args.data_split.split("-")))
	assert sum(args.data_split) == 1, "Sum should be 1"

	# --embed_dim
	assert args.embed_dim in [50, 100, 200, 300], "Embedding dimensions should be from [50, 100, 200, 300]"

	# --embed_name
	assert args.embed_name in ['6B', '42B', '840B'], "Embedding name should be from ['6B', '42B', '840B']"

	if args.embed_name != '6B':
		assert args.embed_dim == 300, "If embedding name is not '6B', then embedding dimension should be 300"

	return args

def makedirs(name):
	"""helper function for python 2 and 3 to call os.makedirs()
		avoiding an error if the directory to be created already exists"""

	import os, errno

	try:
		os.makedirs(name)
	except OSError as ex:
		if ex.errno == errno.EEXIST and os.path.isdir(name):
			# ignore existing directory
			pass
		else:
			# a different error happened
			raise

def check_folder(log_dir):
	if not os.path.exists(log_dir):
		os.makedirs(log_dir)
	return log_dir

def get_logger(args, phase):
	logging.basicConfig(level=logging.INFO, 
												filename = "{}/{}.log".format(args.out_dir, phase),
												format = '%(asctime)s - %(message)s', 
												datefmt='%d-%b-%y %H:%M:%S')
	return logging.getLogger(phase)

# Main

## Arguments

In [6]:
def parse_args():
	parser = ArgumentParser(description='Word Meaning Comparison')
	parser.add_argument('--data_path', '-d', type=str, default='data/train/train.data.txt')
	parser.add_argument('--gold_path', '-l', type=str, default='data/train/train.gold.txt')
	parser.add_argument('--model', '-m', type=str, default='bilstm')

	parser.add_argument('--out_dir', type=str, default='out')
	parser.add_argument('--models_dir', type=str, default='models')
	parser.add_argument('--tmp_dir', type=str, default='tmp')
	
	parser.add_argument('--data_split', type=str, default='1') # eg: '0.8-0.1-0.1', '0.8-0.2', '1'

	parser.add_argument('--gpu', type=int, default=0)
	parser.add_argument('--batch_size', type=int, default=32)
	parser.add_argument('--epochs', type=int, default=25)
	parser.add_argument('--lr', type=float, default=0.001)
	parser.add_argument('--lr_decay', type=float, default=0.0005)	# removed

	parser.add_argument('--embed_name', type=str, default='42B') 
	parser.add_argument('--embed_dim', type=int, default=300) 
	parser.add_argument('--rnn_hidden_dim', type=int, default=150)
	parser.add_argument('--rnn_layers', type=int, default=1)
	parser.add_argument('--dropout', type=float, default=0.5)

	return check_args(parser.parse_known_args()[0])


args = parse_args()
logger = get_logger(args, 'train')
print(args)

Namespace(batch_size=32, data_path='data/train/train.data.txt', data_split=[1.0], dropout=0.5, embed_dim=300, embed_name='42B', epochs=25, gold_path='data/train/train.gold.txt', gpu=0, lr=0.001, lr_decay=0.0005, model='bilstm', models_dir='models', out_dir='out', rnn_hidden_dim=150, rnn_layers=1, tmp_dir='tmp')


In [7]:
# tmp
args.gpu = 1
args.data_split

[1.0]

## Dataset using `torchtext`

In [8]:
label_encode = {
    'F': 0,
    'T': 1
}

label_decode = {
    0: 'F',
    1: 'T'
}

In [9]:
data_path = os.path.join(args.tmp_dir, "data.tsv")

def combine_data():
  data = pd.read_csv(args.data_path, sep='\t', header=None)
  gold = pd.read_csv(args.gold_path, sep='\t', header=None)
  gold[0] = gold.apply(lambda x: label_encode[x[0]], axis=1).astype(int)
  combined = pd.concat([data, gold], axis=1)
  combined['idx_l'] = combined.apply(lambda x: x[2].split('-')[0], axis=1).astype(int)
  combined['idx_r'] = combined.apply(lambda x: x[2].split('-')[-1], axis=1).astype(int)
  combined.columns = ['word', 'pos', 'idx', 's1', 's2', 'label', 'idx_s1', 'idx_s2']
  combined = combined[['word', 'pos', 'idx_s1', 'idx_s2', 's1', 's2', 'label']]
  print(data_path)
  combined.to_csv(data_path, sep='\t', index=False, header=False)
  logger.info("Combined dataset saved at {}".format(data_path))
  return combined

combine_data()

tmp/data.tsv


,word,pos,idx_s1,idx_s2,s1,s2,label
0,carry,V,2,1,You must carry your camping gear .,Sound carries well over water .,0
1,go,V,2,6,Messages must go through diplomatic channels .,Do you think the sofa will go through the door ?,0
2,break,V,0,2,Break an alibi .,The wholesaler broke the container loads into ...,0
3,cup,N,8,4,He wore a jock strap with a metal cup .,Bees filled the waxen cups with honey .,1
4,academy,N,1,2,The Academy of Music .,The French Academy .,0
...,...,...,...,...,...,...,...
5423,krona,N,4,8,Piecas kronas — five krona .,Kronas kurss — the exchange rate of the krona .,1
5424,conflict,N,3,1,The harder the conflict the more glorious the ...,The conflict between the government and the re...,1
5425,answer,V,0,0,Answer the riddle .,Answer a question .,1
5426,play,V,0,0,Play the casinos in Trouville .,Play the races .,1


In [10]:
def str_split(x):
  return x.split()

# WORD = data.Field(lower=True)
# POS = data.Field(lower=True)
INDEX = data.Field(sequential=False, use_vocab=False, batch_first=True)
TEXT = data.Field(sequential=True, tokenize=str_split, lower=True, batch_first=True)
LABEL = data.LabelField(dtype=torch.int, is_target=True, use_vocab=False, batch_first=True)

dataset  = data.TabularDataset(path=data_path, 
                               format='tsv', 
                               fields=[
                                       (None, None), 
                                       (None, None), 
                                       ('idx1', INDEX), 
                                       ('idx2', INDEX), 
                                       ('s1', TEXT), 
                                       ('s2', TEXT), 
                                       ('label', LABEL),
                                       ], skip_header=False)

if len(args.data_split) == 3:
  dt_train, dt_val, dt_test = dataset.split(split_ratio=args.data_split, 
                                            random_state=random.getstate(), 
                                            stratified=True,
                                            strata_field='label',
                                            )
elif len(args.data_split) == 2:
  dt_train, dt_test = dataset.split(split_ratio=args.data_split, 
                                            random_state=random.getstate(), 
                                            stratified=True,
                                            strata_field='label',
                                            )
  dt_val = None
else:
  dt_train, dt_val, dt_test = dataset, None, None
  

glove_embeddings = GloVe(name=args.embed_name, dim=args.embed_dim)


# LABEL.build_vocab(dt_train)


.vector_cache/glove.42B.300d.zip: 1.88GB [05:54, 5.29MB/s]                            
100%|█████████▉| 1917493/1917494 [04:33<00:00, 7003.56it/s]


In [11]:
print([i for i in dt_train.label])
print([i for i in dt_train.s1])

['0', '0', '0', '1', '0', '0', '1', '0', '1', '1', '1', '1', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '1', '1', '1', '0', '0', '0', '1', '1', '1', '0', '0', '0', '1', '0', '0', '1', '0', '1', '1', '0', '0', '1', '0', '1', '0', '1', '1', '1', '0', '1', '0', '0', '1', '0', '0', '0', '1', '0', '0', '1', '0', '1', '0', '0', '1', '1', '1', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '1', '1', '0', '1', '1', '0', '0', '0', '0', '0', '1', '1', '0', '1', '1', '1', '0', '1', '1', '0', '1', '0', '0', '0', '0', '1', '1', '1', '0', '1', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '1', '0', '0', '0', '1', '1', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '1', '0', '0', '0',

In [12]:
glove_vocab = data.Dataset(
    [ data.Example.fromlist(
        [" ".join(glove_embeddings.stoi.keys())],
        [("text", TEXT)]) ], 
    [("text", TEXT)]
)

#print([i for i in glove_vocab.text])
TEXT.build_vocab(glove_vocab, vectors=glove_embeddings)
print ('vector size:',TEXT.vocab.vectors.size())
logger.info("Vocabulary built. Vector size: {}".format(TEXT.vocab.vectors.size()))

vector size: torch.Size([1917496, 300])


In [13]:
device = torch.device('cuda' if args.gpu and torch.cuda.is_available() else 'cpu')

"""
if dt_val is not None and dt_test is not None:
  it_train, it_val, it_test = data.BucketIterator.splits((dt_train, dt_val, dt_test),batch_sizes=(args.batch_size,args.batch_size,args.batch_size), device=device, repeat=False,sort=False, shuffle=True)
if dt_test is not None:
  it_train, it_test = data.BucketIterator.splits((dt_train, dt_test),batch_sizes=(args.batch_size,args.batch_size), device=device, repeat=False,sort=False, shuffle=True)
"""

it_train = data.BucketIterator(dt_train, batch_size=args.batch_size, device=device, repeat=False, sort=False, shuffle=True)
it_test = data.BucketIterator(dt_test, batch_size=len(dt_test), device=device, repeat=False, sort=False, shuffle=False) if dt_test is not None else None
it_val = data.BucketIterator(dt_val, batch_size=len(dt_val), device=device, repeat=False, sort=False, shuffle=False) if dt_val is not None else None

'''
for b in it_test:
    # print (b.s1, b.s2, b.idx1, b.idx2, b.label)
    #print (b.s1)
    #print (b.label)
    print (b.s1.size())
    #print (b.label.size())
    break
'''

'\nfor b in it_test:\n    # print (b.s1, b.s2, b.idx1, b.idx2, b.label)\n    #print (b.s1)\n    #print (b.label)\n    print (b.s1.size())\n    #print (b.label.size())\n    break\n'

## Model

In [14]:
class BiLSTM(nn.Module):
  def __init__(
      self,
      embedding: torch.Tensor,
      rnn_hidden_dim: int,
      rnn_layers: int,
      dropout: float = 0,
      freeze_emb: bool = True
  ) -> None:

    super().__init__()

    assert embedding is not None
    self.embedding = nn.Embedding.from_pretrained(embedding.vectors, freeze=freeze_emb)
    self.emb_size = embedding.vectors.size()[-1]

    self.rnn_hidden_dim = rnn_hidden_dim
    self.encoder = nn.LSTM(
            self.emb_size, rnn_hidden_dim,
            bidirectional = True,
            num_layers = rnn_layers,
            dropout = (0 if rnn_layers == 1 else dropout),
            batch_first = True,
            #proj_size = 50
        )
    self.dropout = nn.Dropout(dropout)
    self.sigmoid = nn.Sigmoid()
    #self.rnn_hidden_dim = 50

    logger.info("New model instantiated with arguments: {}".format(args))

  def forward(self, idx1, idx2, s1, s2):
    batch_size = s1.shape[0]
    t = torch.arange(batch_size)

    s1 = self.embedding(s1)
    s1 = self.dropout(s1)
    s1, _ = self.encoder(s1)
    s1 = s1[ :, :, :self.rnn_hidden_dim ] +  s1[ :, :, self.rnn_hidden_dim: ]
    s1 = s1[t, idx1, :].squeeze(1)

    s2 = self.embedding(s2)
    s2 = self.dropout(s2)
    s2, _ = self.encoder(s2)
    s2 = s2[ :, :, :self.rnn_hidden_dim ] +  s2[ :, :, self.rnn_hidden_dim: ]
    s2 = s2[t, idx2, :].squeeze(1)

    out = torch.sum(s1 * s2, dim=-1)/torch.sqrt(torch.sum(s1**2, dim=-1) * torch.sum(s2**2, dim=-1))
    out = 0.5 * (1 + out)
    ##out = torch.sum(s1 * s2, dim=-1)#.unsqueeze(1)   # dot product
    ## out = self.sigmoid(out) # #todo

    return out


In [15]:
# vocab_size = len(TEXT.vocab)
embedding = TEXT.vocab #.vectors

model = BiLSTM(
    embedding = embedding,
    rnn_hidden_dim = args.rnn_hidden_dim,
    rnn_layers = args.rnn_layers,
    dropout = args.dropout,
    freeze_emb = True
)

#lossf = nn.BCEWithLogitsLoss()
lossf = nn.BCELoss()
#lossf = nn.CrossEntropyLoss()

optimizer = O.Adam(model.parameters(), lr=args.lr) #, weight_decay=args.lr_decay)

model.to(device)

BiLSTM(
  (embedding): Embedding(1917496, 300)
  (encoder): LSTM(300, 150, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (sigmoid): Sigmoid()
)

In [16]:
#scheduler = O.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1, verbose=False)
scheduler = O.lr_scheduler.ExponentialLR(optimizer, gamma=0.96, verbose=False)


In [17]:
def accuracy(y_prob, y_true):
    assert y_true.ndim == 1 and y_true.size() == y_prob.size()
    # y_prob = y_prob > 0.5
    return (y_true == y_prob).sum().item() / y_true.size(0)

def train(model, it, lossf, optimizer):
    model.train()
    ep_loss = 0.0
    ep_acc = 0.0
    for b in tqdm(it):
        optimizer.zero_grad()
        idx1, idx2, s1, s2, label = b.idx1, b.idx2, b.s1, b.s2, b.label
        label = label.float()
        pred = model(idx1, idx2, s1, s2)
        loss = lossf(pred, label)
        acc = accuracy(torch.round(pred), label)
        ##acc = accuracy(torch.round(torch.sigmoid(pred)), label)
        loss.backward()
        #if grad_clip is not None:
        #    clip_gradient(optimizer, grad_clip)
        nn.utils.clip_grad_value_(model.parameters(), 10)
        #nn.utils.clip_grad_norm_(model.parameters(), 10)
        optimizer.step()
        ep_loss += loss.item()
        ep_acc += acc
    return ep_loss/ len(it), ep_acc/ len(it)  # todo: /len(it) ??

def evaluate(model, it, lossf):
  
    model.eval()
    ep_loss = 0.0
    ep_acc = 0.0
    with torch.no_grad():
	    for b in it:
	        idx1, idx2, s1, s2, label = b.idx1, b.idx2, b.s1, b.s2, b.label
	        label = label.float()
	        pred = model(idx1, idx2, s1, s2)
	        loss = lossf(pred, label)
	        ep_loss += loss.item()
	        pred = torch.round(pred)
	        ##pred = torch.round(torch.sigmoid(pred))
	        acc = accuracy(pred, label)
	        ep_acc += acc
    return ep_loss/ len(it), ep_acc/ len(it)


In [18]:
for ep in range(args.epochs):
    print ('training epoch {}... [lr={}]'.format(ep, optimizer.param_groups[0]['lr']))
    logger.info ('Training epoch {}...'.format(ep))
    tr_loss, tr_acc = train(model, it_train, lossf, optimizer)
    print ('TRAIN: loss %.4f acc %.4f' % (tr_loss, tr_acc))
    logger.info ('\tTRAIN: loss %.4f acc %.4f' % (tr_loss, tr_acc))
    if it_val is not None:
      vl_loss, vl_acc = evaluate(model, it_val, lossf)
      print('VALID: loss %.4f acc %.4f' % (vl_loss, vl_acc))
      logger.info('\tVALID: loss %.4f acc %.4f' % (vl_loss, vl_acc))
    scheduler.step()

if it_test is not None:
  te_loss, te_acc = evaluate(model, it_test, lossf)
  print('TEST: loss %.4f acc %.4f' % (te_loss, te_acc))
  logger.info('TEST: loss %.4f acc %.4f' % (te_loss, te_acc))


training epoch 0... [lr=0.001]


100%|██████████| 170/170 [00:02<00:00, 56.96it/s]


TRAIN: loss 0.7116 acc 0.5310
training epoch 1... [lr=0.00096]


100%|██████████| 170/170 [00:02<00:00, 69.49it/s]


TRAIN: loss 0.6631 acc 0.5872
training epoch 2... [lr=0.0009216]


...

TRAIN: loss 0.4141 acc 0.8904
training epoch 23... [lr=0.0003910554653407313]


100%|██████████| 170/170 [00:02<00:00, 70.45it/s]


TRAIN: loss 0.4089 acc 0.9060
training epoch 24... [lr=0.000375413246727102]


100%|██████████| 170/170 [00:02<00:00, 70.61it/s]

TRAIN: loss 0.4000 acc 0.9071


## Store
errors, if any, in this section can be ignored...

In [19]:
if False:
  save_model = {
      'args': args,
      'fields': (INDEX, TEXT, LABEL),
      'model': model,
      'model_state': model.state_dict()
  }

  torch.save(save_model,
            os.path.join(args.out_dir, args.models_dir, "model-{}.pt".format(time.time()))
            )

In [20]:
if False:
  from google.colab import drive
  drive.mount('/content/drive')

In [21]:
"""
!cd out && zip -r ../2018MT10770_A_model.zip models/ && cd ../
!echo "copying..."
!cp "/content/2018MT10770_A_model.zip" "/content/drive/MyDrive/col772_A3/models/"
"""

'\n!cd out && zip -r ../2018MT10770_A_model.zip models/ && cd ../\n!echo "copying..."\n!cp "/content/2018MT10770_A_model.zip" "/content/drive/MyDrive/col772_A3/models/"\n'

In [22]:
# !cp "/content/2018MT10770_A_model.zip" "/content/drive/MyDrive/col772_A3/models/"


In [23]:
# !du -h 2018MT10770_A_model.zip

# Starter Code

In [24]:
## Zip the final model and all the required files, such as vocabulary
# Replace USERID with your own, such as 2017CSZ8058

#!zip -r USERID_A_model.zip **

## Upload it to Google drive and ensure that the testing notebook uses the correct link